In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val rf = new RandomForestClassifier()
    .setNumTrees(10)
    .setMaxDepth(5)
    .setSeed(0)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, rf))

tokenizer = tok_46eb3de7cbc6
hashingTF = hashingTF_f2fbfc084532
rf = rfc_7dedce13f614
pipeline = pipeline_c89aba4b99ea


pipeline_c89aba4b99ea

In [4]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_c89aba4b99ea


pipeline_c89aba4b99ea

In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_c89aba4b99ea


pipeline_c89aba4b99ea

In [7]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[4.65615475151243...|[0.46561547515124...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[5.35540411174777...|[0.53554041117477...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[5.83200854267129...|[0.58320085426712...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[4.75709900618155...|[0.47570990061815...|       1.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[5.48495386141808...|[0.54849538614180...|       0.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [8]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [9]:
predictionsDF.select(getProbability($"probability").alias("clean_probability")).show

+-------------------+
|  clean_probability|
+-------------------+
| 0.5343845248487561|
| 0.4644595888252224|
|0.41679914573287036|
|  0.524290099381844|
|0.45150461385819174|
|0.48106987757033826|
|  0.524290099381844|
|  0.563569885345806|
| 0.5012928518933514|
|  0.524290099381844|
| 0.5168572797762165|
| 0.4915510921355743|
| 0.4326841921301915|
| 0.5235914538920957|
| 0.4340097972344391|
| 0.3812408799529624|
|0.47193994550402163|
| 0.5151601673155273|
| 0.5591803805879312|
|  0.524290099381844|
+-------------------+
only showing top 20 rows

